# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [67]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [68]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mawu,29.5923,107.3139,19.47,71,32,0.96,CN,1697418189
1,1,ribeira grande,38.5167,-28.7000,21.54,91,61,8.94,PT,1697418189
2,2,grytviken,-54.2811,-36.5092,0.34,88,100,4.70,GS,1697418190
3,3,christchurch,-43.5333,172.6333,18.30,40,20,5.14,NZ,1697417544
4,4,poiares,40.2103,-8.2575,19.51,94,100,1.61,PT,1697418190


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
# Configure the map plot
humid_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.6,
    color = "City"    
)

# Display the map plot
humid_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df["Max Temp"] = pd.to_numeric(city_data_df["Max Temp"], errors="coerce")
narrowed_city_df = city_data_df[(city_data_df["Max Temp"].notna()) &
                                (city_data_df["Max Temp"] < 27) &
                                (city_data_df["Max Temp"] > 21) &
                                (city_data_df["Wind Speed"] <= 2.5) &
                                (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
cleaned_narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
cleaned_narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,sirte,31.2089,16.5887,21.91,64,0,1.32,LY,1697418200
62,62,shingu,33.7333,135.9833,22.06,52,0,1.95,JP,1697418211
288,288,iranshahr,27.2025,60.6848,25.19,29,0,0.00,IR,1697418303
323,323,tindouf,27.6711,-8.1474,26.25,22,0,1.54,DZ,1697417797
325,325,susanville,40.4163,-120.6530,21.91,33,0,0.00,US,1697418078
354,354,bandar-e lengeh,26.5579,54.8807,26.06,89,0,0.00,IR,1697418327
397,397,aswan,24.0934,32.9070,23.61,28,0,1.54,EG,1697418343
458,458,tumsar,21.3833,79.7333,21.55,78,0,0.90,IN,1697418366
510,510,gebeit,21.0667,36.3167,21.79,64,0,1.83,SD,1697418388
580,580,turabah,21.2141,41.6331,21.27,49,0,0.88,SA,1697418416


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,sirte,LY,31.2089,16.5887,64,
62,shingu,JP,33.7333,135.9833,52,
288,iranshahr,IR,27.2025,60.6848,29,
323,tindouf,DZ,27.6711,-8.1474,22,
325,susanville,US,40.4163,-120.6530,33,
354,bandar-e lengeh,IR,26.5579,54.8807,89,
397,aswan,EG,24.0934,32.9070,28,
458,tumsar,IN,21.3833,79.7333,78,
510,gebeit,SD,21.0667,36.3167,64,
580,turabah,SA,21.2141,41.6331,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [73]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 10
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        # hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sirte - nearest hotel: فندق المدينة-سرت
shingu - nearest hotel: ホテルニューパレス
iranshahr - nearest hotel: هتل قصِر
tindouf - nearest hotel: محمد بوسبي
susanville - nearest hotel: SureStay Plus Hotel By Best Western Susanville
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel
aswan - nearest hotel: Yaseen hotel
tumsar - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
turabah - nearest hotel: No hotel found
athens - nearest hotel: Eridanus
hasaki - nearest hotel: 大新旅館


,City,Country,Lat,Lng,Humidity,Hotel Name
30,sirte,LY,31.2089,16.5887,64,فندق المدينة-سرت
62,shingu,JP,33.7333,135.9833,52,ホテルニューパレス
288,iranshahr,IR,27.2025,60.6848,29,هتل قصِر
323,tindouf,DZ,27.6711,-8.1474,22,محمد بوسبي
325,susanville,US,40.4163,-120.6530,33,SureStay Plus Hotel By Best Western Susanville
354,bandar-e lengeh,IR,26.5579,54.8807,89,Old owner do not like foreigners hotel
397,aswan,EG,24.0934,32.9070,28,Yaseen hotel
458,tumsar,IN,21.3833,79.7333,78,No hotel found
510,gebeit,SD,21.0667,36.3167,64,No hotel found
580,turabah,SA,21.2141,41.6331,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.6,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)